In [2]:
import numpy as ns
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [3]:
from hello import IEX_CLOUD_API_TOKEN

In [4]:
stocks=pd.read_csv('/content/sp_500_stocks.csv')

In [5]:
final_stocks=pd.DataFrame(columns=['Ticker'])

In [6]:
symbol='AAPL'
api_url=f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data=requests.get(api_url)
print(data.status_code)

200


In [7]:
for stock in stocks['Ticker']:
  api_url=f'https://cloud.iexapis.com/stable/stock/{stock}/stats?token={IEX_CLOUD_API_TOKEN}'
  data=requests.get(api_url)
  if data.status_code==200:
    final_stocks=final_stocks.append(pd.Series([stock],index=['Ticker']),ignore_index=True)

<ipython-input-7-32511dca278a>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_stocks=final_stocks.append(pd.Series([stock],index=['Ticker']),ignore_index=True)
<ipython-input-7-32511dca278a>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_stocks=final_stocks.append(pd.Series([stock],index=['Ticker']),ignore_index=True)
<ipython-input-7-32511dca278a>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_stocks=final_stocks.append(pd.Series([stock],index=['Ticker']),ignore_index=True)
<ipython-input-7-32511dca278a>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_stocks=final_stocks.append(pd.Series([stock],index=['T

In [8]:
def chunks(lst,n):
  for i in range(0,len(lst),n):
    yield lst[i:i+n]
symbol_groups=list(chunks(final_stocks['Ticker'],100))
print(symbol_groups)

[0         A
1       AAL
2       AAP
3      AAPL
4      ABBV
      ...  
95      CLX
96      CMA
97    CMCSA
98      CME
99      CMG
Name: Ticker, Length: 100, dtype: object, 100     CMI
101     CMS
102     CNC
103     CNP
104     COF
       ... 
195    GILD
196     GIS
197      GL
198     GLW
199      GM
Name: Ticker, Length: 100, dtype: object, 200     GOOG
201    GOOGL
202      GPC
203      GPN
204      GPS
       ...  
295      MCK
296      MCO
297     MDLZ
298      MDT
299      MET
Name: Ticker, Length: 100, dtype: object, 300     MGM
301     MHK
302     MKC
303    MKTX
304     MLM
       ... 
395    ROST
396     RSG
397     RTX
398    SBAC
399    SBUX
Name: Ticker, Length: 100, dtype: object, 400    SCHW
401     SEE
402     SHW
403    SIVB
404     SJM
       ... 
491     YUM
492     ZBH
493    ZBRA
494    ZION
495     ZTS
Name: Ticker, Length: 96, dtype: object]


In [9]:
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG',
 'CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS,GL,GLW,GM',
 'GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INFO,IN

In [10]:
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)


In [11]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [12]:
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,price&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol,
                                                   data[symbol]['price'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ],
                                                  index = my_columns),
                                        ignore_index = True)


final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,109.02,-0.193194,N/A
1,AAL,11.98,-0.150957,N/A
2,AAP,57.14,-0.674372,N/A
3,AAPL,176.65,0.285355,N/A
4,ABBV,141.42,0.012755,N/A
...,...,...,...,...
491,YUM,126.34,0.05681,N/A
492,ZBH,108.75,0.045142,N/A
493,ZBRA,214.30,-0.070524,N/A
494,ZION,34.76,-0.269387,N/A


In [13]:
final_dataframe.sort_values('One-Year Price Return',ascending=False,inplace=True)
final_dataframe=final_dataframe[:50]
final_dataframe.reset_index(drop=True,inplace=True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,79.92,2.281287,N/A
1,NVDA,450.05,2.181161,N/A
2,PHM,83.37,1.202357,N/A
3,ADBE,563.66,0.972563,N/A
4,AVGO,882.68,0.957799,N/A
5,FTI,22.25,0.913468,N/A
6,AMD,112.25,0.804953,N/A
7,SNPS,487.94,0.764255,N/A
8,CDNS,249.54,0.752265,N/A
9,GE,108.92,0.712864,N/A


In [14]:
def portfolio_size():
  global portfolio_size
  portfolio_size=input('Enter the size of your portfolio:\n')
  try :
    portfolio_size=float(portfolio_size)
  except:
    portfolio_size=input('Enter the size of your portfolio again in numbers:\n')
    portfolio_size=float(portfolio_size)
portfolio_size()

Enter the size of your portfolio:
1000000


In [15]:
position_size=float(portfolio_size/50)

In [16]:
for i in range(50):
  final_dataframe.loc[i,'Number of Shares to Buy']=math.floor(position_size/final_dataframe.loc[i,'Price'])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,79.92,2.281287,250
1,NVDA,450.05,2.181161,44
2,PHM,83.37,1.202357,239
3,ADBE,563.66,0.972563,35
4,AVGO,882.68,0.957799,22
5,FTI,22.25,0.913468,898
6,AMD,112.25,0.804953,178
7,SNPS,487.94,0.764255,40
8,CDNS,249.54,0.752265,80
9,GE,108.92,0.712864,183


In [17]:
hqm_columns=[
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One Year Price Return',
    'One Year Return Percentile',
    'Six Month Price Return',
    'Six Month Return Percentile',
    'Three Month Price Return',
    'Three Month Return Percentile',
    'One Month Price Return',
    'One Month Return Percentile'
]
hqm_dataframe=pd.DataFrame(columns=hqm_columns)
for symbol_string in symbol_strings:
  batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,price&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([
                                              symbol,
                                              data[symbol]['price'],
                                              'N/A',
                                              data[symbol]['stats']['year1ChangePercent'],
                                              'N/A',
                                              data[symbol]['stats']['month6ChangePercent'],
                                              'N/A',
                                              data[symbol]['stats']['month3ChangePercent'],
                                              'N/A',
                                              data[symbol]['stats']['month1ChangePercent'],
                                              'N/A'
                                                   ],
                                                  index = hqm_columns),
                                        ignore_index = True)


In [18]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile
0,A,109.02,N/A,-0.193194,N/A,-0.182489,N/A,-0.135077,N/A,-0.023731,N/A
1,AAL,11.98,N/A,-0.150957,N/A,-0.113249,N/A,-0.243687,N/A,-0.058916,N/A
2,AAP,57.14,N/A,-0.674372,N/A,-0.520761,N/A,-0.191381,N/A,0.096222,N/A
3,AAPL,176.65,N/A,0.285355,N/A,0.069862,N/A,-0.026719,N/A,0.017218,N/A
4,ABBV,141.42,N/A,0.012755,N/A,-0.019958,N/A,-0.033131,N/A,-0.03287,N/A
...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,126.34,N/A,0.05681,N/A,-0.064523,N/A,-0.055215,N/A,0.022748,N/A
492,ZBH,108.75,N/A,0.045142,N/A,-0.201,N/A,-0.137222,N/A,-0.00857,N/A
493,ZBRA,214.30,N/A,-0.070524,N/A,-0.191321,N/A,-0.147607,N/A,-0.072053,N/A
494,ZION,34.76,N/A,-0.269387,N/A,0.795334,N/A,-0.072693,N/A,0.042904,N/A


In [26]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile
0,A,109.02,N/A,-0.193194,0,-0.182489,0,-0.135077,0,-0.023731,0
1,AAL,11.98,N/A,-0.150957,0,-0.113249,0,-0.243687,0,-0.058916,0
2,AAP,57.14,N/A,-0.674372,0,-0.520761,0,-0.191381,0,0.096222,0
3,AAPL,176.65,N/A,0.285355,0,0.069862,0,-0.026719,0,0.017218,0
4,ABBV,141.42,N/A,0.012755,0,-0.019958,0,-0.033131,0,-0.03287,0
...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,126.34,N/A,0.05681,0,-0.064523,0,-0.055215,0,0.022748,0
492,ZBH,108.75,N/A,0.045142,0,-0.201,0,-0.137222,0,-0.00857,0
493,ZBRA,214.30,N/A,-0.070524,0,-0.191321,0,-0.147607,0,-0.072053,0
494,ZION,34.76,N/A,-0.269387,0,0.795334,0,-0.072693,0,0.042904,0


In [41]:
hqm_dataframe.fillna(0,inplace=True)

In [43]:
time_periods=[
    'One Year',
    'Six Month',
    'Three Month',
    'One Month'
]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])

In [44]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile
0,A,109.02,N/A,-0.193194,14.112903,-0.182489,11.895161,-0.135077,21.169355,-0.023731,24.395161
1,AAL,11.98,N/A,-0.150957,18.548387,-0.113249,21.370968,-0.243687,4.838710,-0.058916,12.096774
2,AAP,57.14,N/A,-0.674372,0.403226,-0.520761,0.403226,-0.191381,8.870968,0.096222,85.887097
3,AAPL,176.65,N/A,0.285355,82.258065,0.069862,61.693548,-0.026719,57.661290,0.017218,47.177419
4,ABBV,141.42,N/A,0.012755,47.379032,-0.019958,41.129032,-0.033131,54.637097,-0.032870,20.362903
...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,126.34,N/A,0.056810,54.435484,-0.064523,30.645161,-0.055215,46.370968,0.022748,50.403226
492,ZBH,108.75,N/A,0.045142,52.217742,-0.201000,10.282258,-0.137222,20.161290,-0.008570,32.459677
493,ZBRA,214.30,N/A,-0.070524,28.629032,-0.191321,11.088710,-0.147607,16.129032,-0.072053,8.467742
494,ZION,34.76,N/A,-0.269387,6.653226,0.795334,99.798387,-0.072693,39.717742,0.042904,59.475806
